# Prepare the Ecologically Important Areas data


### Import libraries and set up environment

In [2]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [3]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

In [ ]:
# Make sure Caribbean_detailed_subregions_mol is in the project (made in EC_study_area_prep)

In [4]:
# Add the EBSA data to the ArcGIS project
# Note that this version of the WBSA data was compiled by Vincent

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.conversion.FeatureClassToGeodatabase(['"O:\f03_centre_initiatives\Protected_Planet_Initiative\0_for_Bex\EBSA_Vincent\V2\338-EBSA_merged_wgs84.shp"'],
                                           output_gdb)


<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb'>

In [4]:
# Add the KBA data (polygons and points) to the ArcGIS project

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.conversion.FeatureClassToGeodatabase([r"O:\f00_data\Birdlife-001-KBA-IBA-AZE\KBAsGlobal_2023_March_01_Criteria\KBAsGlobal_2023_March_01_POL.shp", r"O:\f00_data\Birdlife-001-KBA-IBA-AZE\KBAsGlobal_2023_March_01_Criteria\KBAsGlobal_2023_March_01_PNT.shp"],
                                           output_gdb)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb'>

In [5]:
# Reproject the EBSA data to Mollweide

in_feature = "T338_EBSA_merged_wgs84"
out_feature = "EBSA_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.management.Project(in_feature, out_feature, 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\EBSA_mol'>

In [ ]:
# Calculate the area of the KBA point data in km (it's origianlly in ha)

arcpy.management.CalculateField(
    in_table="KBAsGlobal_2023_March_01_PNT",
    field="Area_km",
    expression="area_km(!sitarea!)",
    expression_type="PYTHON3",
    code_block="""def area_km(sitarea):
    try:
        return float(sitarea) / 100
    except:
        return ""
    
""",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [7]:
# Buffer points of the KBA to be size of Area_km

points = "KBAsGlobal_2023_March_01_PNT"
out_feature = "\\KBA_pt_buf"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

def buffer_points(points):
    #Project to Mollweide (KBA_pt_Mol)
    arcpy.management.Project(points, "KBA_pt_Mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    
    #Calculate radius of buffer 
    arcpy.management.CalculateField("KBA_pt_Mol", "radius", "math.sqrt(!Area_km!/math.pi)*1000", "PYTHON3")

    #buffer to radius.
    #points without area were automatically removed
    arcpy.analysis.Buffer("KBA_pt_Mol", output_gdb + out_feature, "radius", "FULL", "ROUND", "NONE", None, "PLANAR")

    #checked area to make sure it matched (field called area. calc in mollweide, area in sq km)
    arcpy.management.CalculateGeometryAttributes(out_feature, "area AREA", '', "SQUARE_KILOMETERS", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', "SAME_AS_INPUT")

buffer_points(points)

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

...

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'

In [8]:
# Reproject the KBA buffered points to Mollweide

in_feature = "KBA_pt_buf"
out_feature = "KBA_pt_buf_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.management.Project(in_feature, out_feature, 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
    

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\KBA_pt_buf_mol'>

In [9]:
# Append the KBA point layer to the KBA polygon layer

buffered_points = "KBA_pt_buf_mol"
poly = "KBAsGlobal_2023_March_01_POL"
out_feature = "\\KBA_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"


def append_points_to_poly():
    #Project to Mollweide
    arcpy.management.Project(poly, output_gdb + "\\KBA_poly_Mol", 'PROJCS["World_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],UNIT["Meter",1.0]]', None, 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")

    #repair geometry
    arcpy.management.RepairGeometry("KBA_poly_Mol", "DELETE_NULL", "OGC")
    
    #rename it to get ready to append it with points
    arcpy.conversion.ExportFeatures("KBA_poly_Mol", output_gdb + out_feature)
    
    #Append the KBA_mol with the point layer (KBA_pt_buf_mol). 
    #Note that append worked better than merge because merge didn't keep the attributes of the points
    arcpy.management.Append(buffered_points, output_gdb + out_feature, "NO_TEST")
    
    #repair geometry
    arcpy.management.RepairGeometry(out_feature, "DELETE_NULL", "OGC")

append_points_to_poly()

In [ ]:
# Merge the EBSA layer and the KBA layer

buffered_points = "KBA_pt_buf_mol"
poly = "KBAsGlobal_2023_March_01_POL"
out_feature = "\\KBA_EBSA_mol"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.management.Merge(
    inputs="KBA_mol;EBSA_mol",
    output= output_gdb + out_feature,
    field_mappings='SitRecID "SitRecID" true true false 4 Long 0 0,First,#,KBA_mol,SitRecID,-1,-1;Region "Region" true true false 254 Text 0 0,First,#,KBA_mol,Region,0,254;Country "Country" true true false 254 Text 0 0,First,#,KBA_mol,Country,0,254;ISO3 "ISO3" true true false 254 Text 0 0,First,#,KBA_mol,ISO3,0,254;NatName "NatName" true true false 254 Text 0 0,First,#,KBA_mol,NatName,0,254;IntName "IntName" true true false 254 Text 0 0,First,#,KBA_mol,IntName,0,254;FinCode "FinCode" true true false 254 Text 0 0,First,#,KBA_mol,FinCode,0,254;SitLat "SitLat" true true false 8 Double 0 0,First,#,KBA_mol,SitLat,-1,-1;SitLong "SitLong" true true false 8 Double 0 0,First,#,KBA_mol,SitLong,-1,-1;Num_PAs "Num_PAs" true true false 4 Long 0 0,First,#,KBA_mol,Num_PAs,-1,-1;IbaStatus "IbaStatus" true true false 254 Text 0 0,First,#,KBA_mol,IbaStatus,0,254;KbaStatus "KbaStatus" true true false 254 Text 0 0,First,#,KBA_mol,KbaStatus,0,254;AzeStatus "AzeStatus" true true false 254 Text 0 0,First,#,KBA_mol,AzeStatus,0,254;CreatedBy "CreatedBy" true true false 254 Text 0 0,First,#,KBA_mol,CreatedBy,0,254;AddedBy "AddedBy" true true false 254 Text 0 0,First,#,KBA_mol,AddedBy,0,254;AddedDate "AddedDate" true true false 8 Date 0 0,First,#,KBA_mol,AddedDate,-1,-1;ChangeBy "ChangeBy" true true false 254 Text 0 0,First,#,KBA_mol,ChangeBy,0,254;ChangeDate "ChangeDate" true true false 8 Date 0 0,First,#,KBA_mol,ChangeDate,-1,-1;Source "Source" true true false 254 Text 0 0,First,#,KBA_mol,Source,0,254;DelTxt "DelTxt" true true false 254 Text 0 0,First,#,KBA_mol,DelTxt,0,254;DelGeom "DelGeom" true true false 20 Text 0 0,First,#,KBA_mol,DelGeom,0,20;LegacyKBA "LegacyKBA" true true false 254 Text 0 0,First,#,KBA_mol,LegacyKBA,0,254;GISArea "GISArea" true true false 8 Double 0 0,First,#,KBA_mol,GISArea,-1,-1;KBA_Qual "KBA_Qual" true true false 55 Text 0 0,First,#,KBA_mol,KBA_Qual,0,55;Criteria "Criteria" true true false 254 Text 0 0,First,#,KBA_mol,Criteria,0,254;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,KBA_mol,Shape_Length,-1,-1,EBSA_mol,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,KBA_mol,Shape_Area,-1,-1,EBSA_mol,Shape_Area,-1,-1;id "id" true true false 254 Text 0 0,First,#,EBSA_mol,id,0,254;EBSA_ID "EBSA_ID" true true false 4 Long 0 0,First,#,EBSA_mol,EBSA_ID,-1,-1;GLOBAL_ID "GLOBAL_ID" true true false 254 Text 0 0,First,#,EBSA_mol,GLOBAL_ID,0,254;NAME "NAME" true true false 254 Text 0 0,First,#,EBSA_mol,NAME,0,254;WORKSHOP "WORKSHOP" true true false 254 Text 0 0,First,#,EBSA_mol,WORKSHOP,0,254;_OBJECTID "_OBJECTID" true true false 4 Long 0 0,First,#,EBSA_mol,_OBJECTID,-1,-1;_SUBMITTED "_SUBMITTED" true true false 254 Text 0 0,First,#,EBSA_mol,_SUBMITTED,0,254;_Area_MW_K "_Area_MW_K" true true false 8 Double 0 0,First,#,EBSA_mol,_Area_MW_K,-1,-1;_dissolve "_dissolve" true true false 254 Text 0 0,First,#,EBSA_mol,_dissolve,0,254;_NAMEFINAL "_NAMEFINAL" true true false 254 Text 0 0,First,#,EBSA_mol,_NAMEFINAL,0,254;_EBSAFINAL "_EBSAFINAL" true true false 4 Long 0 0,First,#,EBSA_mol,_EBSAFINAL,-1,-1;_INSIDEBND "_INSIDEBND" true true false 254 Text 0 0,First,#,EBSA_mol,_INSIDEBND,0,254;_EBSAREGIO "_EBSAREGIO" true true false 254 Text 0 0,First,#,EBSA_mol,_EBSAREGIO,0,254;_area_laea "_area_laea" true true false 8 Double 0 0,First,#,EBSA_mol,_area_laea,-1,-1;_area_mea "_area_mea" true true false 8 Double 0 0,First,#,EBSA_mol,_area_mea,-1,-1;_area_cea "_area_cea" true true false 4 Long 0 0,First,#,EBSA_mol,_area_cea,-1,-1;_merge "_merge" true true false 254 Text 0 0,First,#,EBSA_mol,_merge,0,254;_layer "_layer" true true false 254 Text 0 0,First,#,EBSA_mol,_layer,0,254;_path "_path" true true false 254 Text 0 0,First,#,EBSA_mol,_path,0,254',
    add_source="NO_SOURCE_INFO"
)

In [10]:
# Clip ecologically important areas data to study area

in_feature = "KBA_EBSA_mol"
clip_feature = "Caribbean_detailed_subregions_mol"
out_feature = "\\KBA_EBSA_mol_Carib"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.Clip(in_feature, clip_feature, out_feature)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\\\KBA_EBSA_mol_Carib'>

In [11]:
# Dissolve the ecologically important areas data
# Make them single part for faster processing

in_feature = "KBA_EBSA_mol_Carib"
out_feature = "\\KBA_EBSA_mol_Carib_dis"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.analysis.PairwiseDissolve(
    in_features= in_feature,
    out_feature_class= output_gdb + out_feature,
    dissolve_field=None,
    statistics_fields=None,
    multi_part="SINGLE_PART",
    concatenation_separator=""
)

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\KBA_EBSA_mol_Carib_dis'>